In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import keras
from keras import backend as K
from keras import initializers
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, Multiply, Reshape, Merge, Flatten, Concatenate
from keras.optimizers import Adam
from keras.regularizers import l2

import json, sys, random, os, datetime, math

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Neural Collaborative Filtering with NeuMF

![An example playist](img/recommended.png)

An example of the recommendations for my Spotify R&B playlist.

I am going to use only 1/100 of the data (10,000 sample playlists) for training due to time and resource constraints. This approach should scale if you have a server at your disposal. The NeuMF model for neural collaborative filtering combines both GMF and MLP (neural net) approaches. 

### Architecture for the NeuMF model:
![NeuMF Architecture](img/neumf.png)
Image taken from [He et al, 2017](https://arxiv.org/pdf/1708.05031.pdf), who developed this model and describe the approach in greater detail.

### Steps
1. Build sparse one-hot encoding matrix that maps relationships between users (playlists) and items (tracks).
2. Playlist (more generally called u for user) and item (i) vectors are used to create embeddings (low-dimensional) for each playlist and item.
3. Generalized Matrix Factorization (GMF) combines the two embeddings using the dot product (this is the classic matrix factorization).
4. Multi-layer perceptron (MLP) can also create embeddings for user and items. However, instead of taking a dot product of these to obtain the rating, I can concatenate them to create a feature vector that is passed on to deeper layers.
5. NeuMF then combines the predictions from MLP and GMF to obtain the final prediction.

### Data preprocessing

Build sparse one-hot encoding matrix that maps relationships between users (playlists) and items (tracks).

In [2]:
%%time
def make_playlist_df(path, num_slices):
    df_list = []
    count = 0
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if count < num_slices and filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            data = json.load(open(fullpath))
            slice_df = pd.DataFrame.from_dict(data['playlists'], orient='columns')
            df_list.append(slice_df)
            count += 1
    return pd.concat(df_list)

path = '../mpd.v1/data'
traindata = make_playlist_df(path, 10)

CPU times: user 2.14 s, sys: 399 ms, total: 2.54 s
Wall time: 2.61 s


In [3]:
# also read in the challenge dataset which has missing songs
# that I want the model to predict
t = json.load(open('../challenge.v1/challenge_set.json'))
challenge_df = pd.DataFrame.from_dict(t['playlists'], orient='columns')

# combine train and challenge so can use cat code to map
# track ids to an index 0-N across both datasets
train_challengedata = pd.concat([traindata, challenge_df])

In [4]:
# turn playlist level dataframe into song level dataframe
complete_array = []
for index, row in train_challengedata.iterrows():
    for track in row['tracks']:
        complete_array.append([track['track_uri'], track['artist_name'], track['track_name'], row['pid'], row['num_holdouts']])
df = pd.DataFrame(complete_array, columns=['trackid', 'artist_name', 'track_name', 'pid', 'num_holdouts'])

print(df.shape)
df.head()   # is a df of all track ids, corresponding artist names, track names and playlist ids

(951568, 5)


,trackid,artist_name,track_name,pid,num_holdouts
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0,NaN
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,0,NaN
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,0,NaN
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,0,NaN
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,0,NaN


In [5]:
# turn songs into their unique cat codes so have a 0-N index for tracks
df['trackindex'] = df['trackid'].astype('category').cat.codes
print(len(df['trackindex'].unique()))
df.head()

189359


,trackid,artist_name,track_name,pid,num_holdouts,trackindex
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0,NaN,12216
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,0,NaN,153108
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,0,NaN,13148
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,0,NaN,28658
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,0,NaN,43317


In [6]:
# split training and challenge data
train = df[pd.isnull(df['num_holdouts'])]
challenge = df[pd.notnull(df['num_holdouts'])]
train.head()

,trackid,artist_name,track_name,pid,num_holdouts,trackindex
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0,NaN,12216
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,0,NaN,153108
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,0,NaN,13148
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,0,NaN,28658
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,0,NaN,43317


In [7]:
challenge.head()

,trackid,artist_name,track_name,pid,num_holdouts,trackindex
670568,spotify:track:66U0ASk1VHZsqIkpMjKX3B,AronChupa,Little Swing,1000000,70.0,148529
670569,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,AronChupa,I'm an Albatraoz,1000000,70.0,130788
670570,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,Lorde,Yellow Flicker Beat - From The Hunger Games: M...,1000000,70.0,6745
670571,spotify:track:35kahykNu00FPysz3C2euR,Lorde,White Teeth Teens,1000000,70.0,75377
670572,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,Lorde,Team,1000000,70.0,79446


In [8]:
%%time
# save data in dok matrix (optimized sparse matrix object)
# create a sparse playlistid x trackindex matrix
# if a playlistid i has song j, mat[i,j]=1
mat = sp.dok_matrix((train.shape[0], len(df['trackindex'].unique())), dtype=np.float32)
for pid, trackindex in zip(train['pid'], train['trackindex']):
    mat[pid, trackindex] = 1.0
# sp.save_npz('spotify_train_matrix.npz', mat)

CPU times: user 5.23 s, sys: 52.1 ms, total: 5.28 s
Wall time: 5.28 s


### Training the model

Now that we have a sparse user-item encoding matrix, we can set up the NeuMF model. See the diagram above for details on model architecture. Note that I generate negative training examples (playlist-track pairs that are not valid) since there is a natural scarcity of negative feedback in playlist recommendation. We only know which tracks are in each playlists, not explicitly which tracks are not. I use a tower architecture (dense layers decrease in size as you move deeper into the model) because this allows the model to learn nore abstract features. 

Note that I train for 5 epochs to improve accuracy. However, since I trained for several epochs, I could only train on 10,000 samples due to time and resource constraints. This model will scale to the full million samples given more time/resources (perhaps hosted through AWS?), but I just didn't have time to train all of that. 

A quick comparison with my GMF-only model (you can see this in matrix_factorization.ipynb): I was able to train this with 10 times the amount of data (because GMF-only runs **much** faster than the full NeuMF model), and I got an accuracy of 0.9 after training on 3 epochs. Interestingly, my complete NeuMF model acheived an accuracy of 0.9293 after only 3 epochs, and on only 1/10th of that data! The nonlinear user-item interactions learned by the MLP component significantly boost performance, though I would have liked to train with more data to help guard against overfitting.

In [9]:
# full NCF model
def get_model(num_users, num_items, latent_dim=8, dense_layers=[64, 32, 16, 8],
              reg_layers=[0, 0, 0, 0], reg_mf=0):

    # input layer
    input_user = Input(shape=(1,), dtype='int32', name='user_input')
    input_item = Input(shape=(1,), dtype='int32', name='item_input')
    
    # embedding layer
    mf_user_embedding = Embedding(input_dim=num_users, output_dim=latent_dim,
                        name='mf_user_embedding',
                        embeddings_initializer='RandomNormal',
                        embeddings_regularizer=l2(reg_mf), input_length=1)
    mf_item_embedding = Embedding(input_dim=num_items, output_dim=latent_dim,
                        name='mf_item_embedding',
                        embeddings_initializer='RandomNormal',
                        embeddings_regularizer=l2(reg_mf), input_length=1)
    mlp_user_embedding = Embedding(input_dim=num_users, output_dim=int(dense_layers[0]/2),
                         name='mlp_user_embedding',
                         embeddings_initializer='RandomNormal',
                         embeddings_regularizer=l2(reg_layers[0]), 
                         input_length=1)
    mlp_item_embedding = Embedding(input_dim=num_items, output_dim=int(dense_layers[0]/2),
                         name='mlp_item_embedding',
                         embeddings_initializer='RandomNormal',
                         embeddings_regularizer=l2(reg_layers[0]), 
                         input_length=1)

    # MF latent vector
    mf_user_latent = Flatten()(mf_user_embedding(input_user))
    mf_item_latent = Flatten()(mf_item_embedding(input_item))
    mf_cat_latent = Multiply()([mf_user_latent, mf_item_latent])

    # MLP latent vector
    mlp_user_latent = Flatten()(mlp_user_embedding(input_user))
    mlp_item_latent = Flatten()(mlp_item_embedding(input_item))
    mlp_cat_latent = Concatenate()([mlp_user_latent, mlp_item_latent])
    
    mlp_vector = mlp_cat_latent
    
    # build dense layer for model
    for i in range(1,len(dense_layers)):
        layer = Dense(dense_layers[i],
                      activity_regularizer=l2(reg_layers[i]),
                      activation='relu',
                      name='layer%d' % i)
        mlp_vector = layer(mlp_vector)

    predict_layer = Concatenate()([mf_cat_latent, mlp_vector])
    result = Dense(1, activation='sigmoid', 
                   kernel_initializer='lecun_uniform',name='result')

    model = Model(inputs=[input_user,input_item], outputs=result(predict_layer))

    return model

# get the training samples
def get_train_samples(train_mat, num_negatives):
    user_input, item_input, labels = [], [], []
    num_user, num_item = train_mat.shape
    for (u, i) in train_mat.keys():
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_item)
            while (u, j) in train_mat.keys():
                j = np.random.randint(num_item)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

# hyperparameters
loaded = True
verbose = 1
epochs = 5   # I'd prefer to do 10, but time constraints
batch_size = 256
latent_dim = 8
dense_layers = [64, 32, 16, 8]
reg_layers = [0, 0, 0, 0]
reg_mf = [0]
num_negatives = 4
learning_rate = 0.001
learner = 'adam'
dataset = 'spotify'

# loading data
if loaded:
    train_mat = mat
else:
    train_mat = sp.load_npz('spotify_train_matrix.npz')
    
num_users, num_items = train_mat.shape
print('Done loading data!')

Done loading data!


In [10]:
%%time

# get model
model = get_model(num_users, num_items, latent_dim, dense_layers, reg_layers, reg_mf)
model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())
    
# train model
# generate training instances
user_input, item_input, labels = get_train_samples(train_mat, num_negatives)

# training
hist = model.fit([np.array(user_input), np.array(item_input)], np.array(labels), 
                 batch_size=batch_size, epochs=epochs, verbose=verbose, shuffle=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
mlp_user_embedding (Embedding)  (None, 1, 32)        21458176    user_input[0][0]                 
__________________________________________________________________________________________________
mlp_item_embedding (Embedding)  (None, 1, 32)        6059488     item_input[0][0]                 
__________________________________________________________________________________________________
flatten_3 

In [13]:
# save model
model_file = '%s_NCF_%d_%s.h5' % (dataset, latent_dim, str(dense_layers))
model.save(model_file, overwrite=True)

### Recommending songs for a playlist

Now that I've trained my model, I'm ready to recommend songs for a given playlist! However, one issue that I encountered (see below) is that I need the embedding of that new playlist (as stored in my model) in order to find the closest relevant playlists in that embedding space using kmeans. I am not sure how to get around this issue- as is, it seems that I have to retrain my whole model each time I get an input playlist in order to get that playlist embedding. Therefore, I just test my model on a randomly chosen playlist (which happens to be rock and oldies, mostly!) from the training set. 

To recommend songs, I first cluster the learned embeddings for all of the training playlists, and then select "neighbor" playlists for my given test playlist as all of the other playlists in that same cluster. I then take all of the tracks from these playlists and feed the test playlist embedding and these "neighboring" tracks into my model for prediction. This ranks the "neighboring" tracks by how likely they are (under my model) to occur next in the given test playlist. 

In [24]:
%%time
from keras.models import load_model
from sklearn.cluster import KMeans

# this is a nice rock/oldies playlist
desired_user_id = 500
model_path = 'spotify_NCF_8_[64, 32, 16, 8].h5'
print('using model: %s' % model_path)
model = load_model(model_path)
print('Loaded model!')

mlp_user_embedding_weights = (next(iter(filter(lambda x: x.name == 'mlp_user_embedding', model.layers))).get_weights())

# get the latent embedding for your desired user
user_latent_matrix = mlp_user_embedding_weights[0]
one_user_vector = user_latent_matrix[desired_user_id,:]
one_user_vector = np.reshape(one_user_vector, (1,32))

print('\nPerforming kmeans to find the nearest users/playlists...')
# get 100 similar users
kmeans = KMeans(n_clusters=100, random_state=0, verbose=0).fit(user_latent_matrix)
desired_user_label = kmeans.predict(one_user_vector)
user_label = kmeans.labels_
neighbors = []
for user_id, user_label in enumerate(user_label):
    if user_label == desired_user_label:
        neighbors.append(user_id)
print('Found {0} neighbor users/playlists.'.format(len(neighbors))) 

using model: spotify_NCF_8_[64, 32, 16, 8].h5
Loaded model!

Performing kmeans to find the nearest users/playlists...
Found 359 neighbor users/playlists.
CPU times: user 23min 37s, sys: 10min 6s, total: 33min 43s
Wall time: 33min 24s


In [45]:
# get the tracks in similar users' playlists
tracks = []
for user_id in neighbors:
    tracks += list(df[df['pid'] == int(user_id)]['trackindex'])
print('Found {0} neighbor tracks from these users.'.format(len(tracks))) 

users = np.full(len(tracks), desired_user_id, dtype='int32')
items = np.array(tracks, dtype='int32')

print('\nRanking most likely tracks using the NeuMF model...')
# and predict tracks for my user
results = model.predict([users,items],batch_size=100, verbose=0) 
results = results.tolist()
print('Ranked the tracks!')

Found 19285 neighbor tracks from these users.

Ranking most likely tracks using the NeuMF model...
Ranked the tracks!


In [66]:
results_df = pd.DataFrame(np.nan, index=range(len(results)), columns=['probability','track_name', 'track artist'])
print(results_df.shape)

# loop through and get the probability (of being in the playlist according to my model), the track, and the track's artist 
for i, prob in enumerate(results):
    results_df.loc[i] = [prob[0], df[df['trackindex'] == i].iloc[0]['track_name'], df[df['trackindex'] == i].iloc[0]['artist_name']]
results_df = results_df.sort_values(by=['probability'], ascending=False)

results_df.head(20)

(19285, 3)


,probability,track_name,track artist
11774,1.000000,Nadie,La Mafia
8150,1.000000,Follow The Leader,Eric B. & Rakim
9341,1.000000,Champagne,Kid Indigo
971,0.999999,All I Want For Christmas,TLC
3784,0.999999,Looking For Cider,Alexandre Desplat
7079,0.999999,Old Fashioned,William Clark Green
14174,0.999999,Lessons,Naomi Wild
17314,0.999999,Running,Kid Luna
9788,0.999999,Asi Naci,Alta Consigna
402,0.999999,Brightened Faces,Mike Edel


Above you can see the 20 "most likely" songs recommended by my model. Below, you can see 20 songs from the actual test playlist. tl;dr: they're pretty different. My model recommends a lot of Latin dance music (Shakira, RKM, Richie Ray) for a classic rock/oldies playlist. I wonder if my model has overfit, but I think that I definitely didn't train it on enough samples.

In [22]:
df[df['pid'] == 500].head(20)

,trackid,artist_name,track_name,pid,num_holdouts,trackindex
33415,spotify:track:2H3ZUSE54pST4ubRd5FzFR,Marvin Gaye,Ain't No Mountain High Enough,500,NaN,55489
33416,spotify:track:1QEEqeFIZktqIpPI4jSVSF,Boston,More Than a Feeling,500,NaN,34827
33417,spotify:track:7BY005dacJkbO6EPiOh2wb,The Animals,House Of The Rising Sun,500,NaN,174830
33418,spotify:track:43btz2xjMKpcmjkuRsvxyg,John Mellencamp,Jack & Diane,500,NaN,98805
33419,spotify:track:00MI0oGDVJYM1qWbyUOIhH,Tommy Tutone,867-5309 / Jenny,500,NaN,148
33420,spotify:track:6N1EjQjnvhOjFrF6oUmGPa,Bryan Adams,Summer Of '69,500,NaN,154990
33421,spotify:track:3vV3cr2TpPqFk07zxYUbla,Quiet Riot,Cum on Feel the Noize,500,NaN,95644
33422,spotify:track:0XIvZ82aDF7JiSi3ZE320u,Scorpions,Rock You Like A Hurricane,500,NaN,13331
33423,spotify:track:5tVA6TkbaAH9QMITTQRrNv,Tom Petty,Free Fallin',500,NaN,143462
33424,spotify:track:63OFKbMaZSDZ4wtesuuq6f,Steppenwolf,Born To Be Wild,500,NaN,147302
